# Implementing Advantage-Actor Critic (A2C) - 2 pts

In this notebook you will implement Advantage Actor Critic algorithm that trains on a batch of Atari 2600 environments running in parallel. 

Firstly, we will use environment wrappers implemented in file `atari_wrappers.py`. These wrappers preprocess observations (resize, grayscal, take max between frames, skip frames, stack them together, prepares for PyTorch and normalizes to [0, 1]) and rewards. Some of the wrappers help to reset the environment and pass `done` flag equal to `True` when agent dies.
File `env_batch.py` includes implementation of `ParallelEnvBatch` class that allows to run multiple environments in parallel. To create an environment we can use `nature_dqn_env` function.

In [1]:
import numpy as np
from atari_wrappers import nature_dqn_env

nenvs = 8    # change this if you have more than 8 CPU ;)

env = nature_dqn_env("SpaceInvadersNoFrameskip-v4", nenvs=nenvs)

n_actions = env.action_space.spaces[0].n
obs = env.reset()
assert obs.shape == (nenvs, 4, 84, 84)
assert obs.dtype == np.float32

Next, we will need to implement a model that predicts logits of policy distribution and critic value. Use shared backbone. You may use same architecture as in DQN task with one modification: instead of having a single output layer, it must have two output layers taking as input the output of the last hidden layer (one for actor, one for critic). 

Still it may be very helpful to make more changes:
* use orthogonal initialization with gain $\sqrt{2}$ and initialize biases with zeros;
* use more filters (e.g. 32-64-64 instead of 16-32-64);
* use two-layer heads for actor and critic or add a linear layer into backbone;

**Danger:** do not divide on 255, input is already normalized to [0, 1] in our wrappers!

In [2]:
import random
import numpy as np
import torch
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
from torch.nn import Flatten
from torch.distributions import Categorical
%matplotlib inline

In [3]:
class Model(nn.Module):
    def __init__(self, n_actions):

        super().__init__()
        self.n_actions = n_actions

        # Define your network body here. Please make sure agent is fully contained here
        
        
        self.conv_layers = nn.Sequential(nn.Conv2d(4,32,(3,3),stride=2),nn.ReLU(),
                                         nn.Conv2d(32,64,(3,3),stride=2), nn.ReLU(),
                                         nn.Conv2d(64,64,(3,3),stride=2),nn.ReLU())
        self.hidden = nn.Linear(5184,200)
        self.linear = nn.Linear(200,self.n_actions)
        self.linear_val = nn.Linear(200,1)
        self.softmax = nn.Softmax()
        
        for layer in self.conv_layers:
            if isinstance(layer,nn.Conv2d):
                nn.init.orthogonal_(layer.weight, gain=np.sqrt(2))
                nn.init.constant_(layer.bias, 0)
            
        nn.init.orthogonal_(self.hidden.weight, gain=np.sqrt(2))
        nn.init.orthogonal_(self.linear.weight, gain=np.sqrt(2))
        nn.init.orthogonal_(self.linear_val.weight, gain=np.sqrt(2))
        
        nn.init.constant_(self.hidden.bias, 0)
        nn.init.constant_(self.linear.bias, 0)
        nn.init.constant_(self.linear_val.bias, 0)
        
        
    def forward(self, state_t):
        """
        takes agent's observation (tensor), returns qvalues (tensor)
        :param state_t: a batch of 4-frame buffers, shape = [batch_size, 4, h, w]
        """
        # Use your network to compute qvalues for given state
        conv = self.conv_layers(state_t)
        hidden = self.hidden(Flatten()(conv))
        probs = self.linear(hidden)
        value = self.linear_val(hidden)
        return self.softmax(probs), value



You will also need to define and use a policy that wraps the model. While the model computes logits for all actions, the policy will sample actions and also compute their log probabilities.  `policy.act` should return a **dictionary** of all the arrays that are needed to interact with an environment and train the model.

**Important**: "actions" will be sent to environment, they must be numpy array or list, not PyTorch tensor.

Note: you can add more keys, e.g. it can be convenient to compute entropy right here.

In [4]:
from torch.distributions import Categorical
from collections import defaultdict
class Policy:
    def __init__(self, model):
        self.model = model

    def act(self, inputs):
        '''
        input:|
            inputs - numpy array, (batch_size x channels x width x height)
        output: dict containing keys ['actions', 'logits', 'log_probs', 'values']:
            'actions' - selected actions, numpy, (batch_size)
            'logits' - actions logits, tensor, (batch_size x num_actions)
            'log_probs' - log probs of selected actions, tensor, (batch_size)
            'values' - critic estimations, tensor, (batch_size)
        '''
        logits, values = self.model(torch.tensor(inputs))
        actions = []
        for row in logits:
            probs =(row.detach()/row.sum().detach()).numpy()
            actions.append(np.random.choice(range(6),p=probs))

        actions = np.array(actions)

        log_probs = torch.log(logits[list(range(len(logits))),actions])
        
        entropy = (logits*torch.log(logits)).sum(1)

        return {
            "actions": actions,
            "logits": logits,
            "log_probs": log_probs,
            "values": values.view(-1),
            "entropy":entropy,
        }

Next we will pass the environment and policy to a runner that collects rollouts from the environment. 
The class is already implemented for you.

In [5]:
from runners import EnvRunner
import random

This runner interacts with the environment for a given number of steps and returns a dictionary containing
keys 

* 'observations' 
* 'rewards' 
* 'dones'
* 'actions'
* all other keys that you defined in `Policy`

under each of these keys there is a python `list` of interactions with the environment of specified length $T$ &mdash; the size of partial trajectory, or rollout length. Let's have a look at how it works.

In [6]:
model = Model(n_actions)
policy = Policy(model)
runner = EnvRunner(env, policy, nsteps=5)

In [7]:
# generates new rollout
trajectory = runner.get_next()

C:\Users\user\AppData\Local\Temp\ipykernel_10144\2481146560.py:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.softmax(probs), value


In [8]:
# what is inside
print(trajectory.keys())

dict_keys(['actions', 'logits', 'log_probs', 'values', 'entropy', 'observations', 'rewards', 'dones'])


For each key the length of the corresponding list is the number of steps. The size of each items is the size of the batch.

In [9]:
# Sanity checks
assert 'logits' in trajectory, "Not found: policy didn't provide logits"
assert 'log_probs' in trajectory, "Not found: policy didn't provide log_probs of selected actions"
assert 'values' in trajectory, "Not found: policy didn't provide critic estimations"
assert trajectory['logits'][0].shape == (nenvs, n_actions), "logits wrong shape"
assert trajectory['log_probs'][0].shape == (nenvs,), "log_probs wrong shape"
assert trajectory['values'][0].shape == (nenvs,), "values wrong shape"

for key in trajectory.keys():
    assert len(trajectory[key]) == 5, \
    f"something went wrong: 5 steps should have been done, got trajectory of length {len(trajectory[key])} for '{key}'"

Now let's work with this trajectory a bit. To train the critic you will need to compute the value targets. It will also be used as an estimation of $Q$ for actor training.

You should use all available rewards for value targets, so the formula for the value targets is simple:

$$
\hat v(s_t) = \sum_{t'=0}^{T - 1}\gamma^{t'}r_{t+t'} + \gamma^T \hat{v}(s_{t+T}),
$$

where $s_{t + T}$ is the latest observation of the environment.

Any callable could be passed to `EnvRunner` to be applied to each partial trajectory after it is collected. 
Thus, we can implement and use `ComputeValueTargets` callable. 

**Do not forget** to use `trajectory['dones']` flags to check if you need to add the value targets at the next step when 
computing value targets for the current step.

**Bonus (+0.5 pts):** implement [Generalized Advantage Estimation (GAE)](https://arxiv.org/pdf/1506.02438.pdf) instead; use $\lambda \approx 0.95$ or even closer to 1 in experiment. 

In [10]:
class ComputeValueTargets:
    def __init__(self, policy, gamma=0.99):
        self.policy = policy
        self.gamma = gamma

    def __call__(self, trajectory, latest_observation):
        '''
        This method should modify trajectory inplace by adding 
        an item with key 'value_targets' to it
        
        input:
            trajectory - dict from runner
            latest_observation - last state, numpy, (num_envs x channels x width x height)
        '''
        last_value = self.policy.act(latest_observation)["values"].detach().cpu().numpy()
        value_targets = []
        G_curr = last_value
        T = len(trajectory["rewards"])
        for i in range(T-1,-1,-1):
            G_curr = trajectory["rewards"][i] + self.gamma * G_curr * (1-trajectory["dones"][i])
            value_targets.append(torch.tensor(G_curr))
        value_targets.reverse()
        trajectory['value_targets'] = value_targets

After computing value targets we will transform lists of interactions into tensors
with the first dimension `batch_size` which is equal to `T * nenvs`.

You need to make sure that after this transformation `"log_probs"`, `"value_targets"`, `"values"` are 1-dimensional PyTorch tensors.

In [11]:
class MergeTimeBatch:
    """ Merges first two axes typically representing time and env batch. """
    def __call__(self, trajectory, latest_observation):
        # Modify trajectory inplace. 
        trajectory["log_probs"] = torch.stack(trajectory["log_probs"]).view(-1)
        trajectory["value_targets"] = torch.stack(trajectory["value_targets"]).view(-1)
        trajectory["values"] = torch.stack(trajectory["values"]).view(-1)

Let's do more sanity checks!

In [12]:
runner = EnvRunner(env, policy, nsteps=5, transforms=[ComputeValueTargets(policy),
                                                      MergeTimeBatch()])

trajectory = runner.get_next()

C:\Users\user\AppData\Local\Temp\ipykernel_10144\2481146560.py:42: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self.softmax(probs), value


In [13]:
# More sanity checks
assert 'value_targets' in trajectory, "Value targets not found"
assert trajectory['log_probs'].shape == (5 * nenvs,)
assert trajectory['value_targets'].shape == (5 * nenvs,)
assert trajectory['values'].shape == (5 * nenvs,)

assert trajectory['log_probs'].requires_grad, "Gradients are not available for actor head!"
assert trajectory['values'].requires_grad, "Gradients are not available for critic head!"

In [14]:
trajectory['values'].detach()

tensor([0.2220, 0.2220, 0.2220, 0.2220, 0.2220, 0.2220, 0.2220, 0.2220, 0.2382,
        0.2382, 0.2382, 0.2382, 0.2382, 0.2382, 0.2382, 0.2382, 0.2170, 0.2170,
        0.2170, 0.2170, 0.2170, 0.2170, 0.2170, 0.2170, 0.2037, 0.2037, 0.2037,
        0.2037, 0.2037, 0.2037, 0.2037, 0.2037, 0.2269, 0.2269, 0.2269, 0.2269,
        0.2269, 0.2269, 0.2269, 0.2269])

Now is the time to implement the advantage actor critic algorithm itself. You can look into [Mnih et al. 2016](https://arxiv.org/abs/1602.01783) paper, and lectures ([part 1](https://www.youtube.com/watch?v=Ds1trXd6pos&list=PLkFD6_40KJIwhWJpGazJ9VSj9CFMkb79A&index=5), [part 2](https://www.youtube.com/watch?v=EKqxumCuAAY&list=PLkFD6_40KJIwhWJpGazJ9VSj9CFMkb79A&index=6)) by Sergey Levine.

In [15]:
from collections import defaultdict
from torch.nn.utils import clip_grad_norm_

class A2C:
    def __init__(self, policy, optimizer, value_loss_coef=0.25, entropy_coef=0.01, max_grad_norm=0.5):
        self.policy = policy
        self.optimizer = optimizer
        self.value_loss_coef = value_loss_coef
        self.entropy_coef = entropy_coef
        self.max_grad_norm = max_grad_norm
    
    def loss(self, trajectory, write):
        policy_loss = (trajectory['log_probs'] * (trajectory['values'].detach() - trajectory['value_targets'])).mean()
        target_loss = ((trajectory['values'] - trajectory['value_targets'])**2).mean()
        entropy_loss = torch.stack(trajectory["entropy"]).view(-1).mean()

        write('losses', {
            'policy loss': policy_loss,
            'critic_loss': target_loss,
            'entropy_loss': entropy_loss
        })
        
        # additional logs
        write('critic/advantage', (trajectory["values"] - trajectory["value_targets"]).mean())
        write('critic/values', {
            'value predictions': trajectory["values"].mean(),
            'value targets': trajectory["value_targets"].mean(),
        })
        

        return policy_loss + target_loss * self.value_loss_coef +  entropy_loss * self.entropy_coef 

    def train(self, runner):
        trajectory = runner.get_next()
        loss = self.loss(trajectory,runner.write)
        self.optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(self.policy.model.parameters(),self.max_grad_norm)
        optimizer.step()

        # use runner.write to log scalar to tensorboard
#         runner.write('gradient norm', )


Now you can train your model. For optimization we suggest you use RMSProp with learning rate 7e-4 (you can also linearly decay it to 0), smoothing constant (alpha in PyTorch) equal to 0.99 and epsilon equal to 1e-5.

We recommend to train for at least 10 million environment steps across all batched environments (takes ~3 hours on a single GTX1080 with 8 CPU). It should be possible to achieve *average raw reward over last 100 episodes* (the average is taken over 100 last episodes in each environment in the batch) of about 600. **Your goal is to reach 500**.

Notes:
* if your reward is stuck at ~200 for more than 2M steps then probably there is a bug
* if your gradient norm is >10 something probably went wrong
* make sure your `entropy loss` is negative, your `critic loss` is positive
* make sure you didn't forget `.detach` in losses where it's needed
* `actor loss` should oscillate around zero or near it; do not expect loss to decrease in RL ;)
* you can experiment with `nsteps` ("rollout length"); standard rollout length is 5 or 10. Note that this parameter influences how many algorithm iterations is required to train on 10M steps (or 40M frames --- we used frameskip in preprocessing).

In [16]:
from tqdm import tqdm
from IPython.display import clear_output

In [17]:
model = Model(6)
policy = Policy(model)
runner = EnvRunner(env, policy, nsteps=10, transforms=[ComputeValueTargets(policy),
                                                      MergeTimeBatch()])

optimizer = torch.optim.RMSprop(model.parameters(),lr=7e-4,eps=1e-5)

a2c = A2C(policy,optimizer,)

In [18]:
# dct = torch.load("modA2C_30000")
# model.load_state_dict(dct)

In [19]:
def compute_score():
    env = nature_dqn_env("SpaceInvadersNoFrameskip-v4", nenvs=None, 
                     clip_reward=False, summaries=False, episodic_life=False)
    sessions = evaluate(env, policy, n_games=30)
    score = sessions.mean()
    return score

In [20]:
def evaluate(env, policy, n_games=1, t_max=10000):
    '''
    Plays n_games and returns rewards
    '''
    rewards = []
    
    for _ in range(n_games):
        s = env.reset()
        
        R = 0
        for _ in range(t_max):
            action = policy.act(np.array([s]))["actions"][0]
            
            s, r, done, _ = env.step(action)
            
            R += r
            if done:
                break

        rewards.append(R)
    return np.array(rewards)

In [21]:
for i in tqdm(range(4*10**4)):
    a2c.train(runner)
    if i % 10000 == 0:
#         print(i,compute_score())
        torch.save(model.state_dict(), f"modA2C_{i}")    

In [22]:
env.close()

## Evaluation

In [23]:
env = nature_dqn_env("SpaceInvadersNoFrameskip-v4", nenvs=None, 
                     clip_reward=False, summaries=False, episodic_life=False)

In [25]:
# evaluation will take some time!
sessions = evaluate(env, policy, n_games=30)
score = sessions.mean()
print(f"Your score: {score}")

assert score >= 500, "Needs more training?"
print("Well done!")

Your score: 528.6666666666666
Well done!


In [26]:
env.close()

## Record

In [39]:
env_monitor = nature_dqn_env("SpaceInvadersNoFrameskip-v4", nenvs=None, monitor=True,
                             clip_reward=False, summaries=False, episodic_life=False)

In [40]:
# record sessions
sessions = evaluate(env_monitor, policy, n_games=3)

In [41]:
# rewards for recorded games
sessions

array([980., 650., 445.])

In [42]:
env_monitor.close()